
## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

We will focus specifically on Shakespeare's Sonnets in order to improve our model's ability to learn from the data.

In [93]:
from google.colab import files
files.upload() 

Saving data_cleaning_toolkit_class.py to data_cleaning_toolkit_class.py


{'data_cleaning_toolkit_class.py': b'import re\r\nimport numpy as np\r\n\r\n\r\nclass data_cleaning_toolkit(object):\r\n    \r\n    def __init__(self):\r\n        """\r\n        This class host several data cleaning and preperation methods that are used to prepare text data \r\n        for a text generation model, specifically for the LSTM.\r\n        """\r\n        \r\n        self.sequences = None\r\n        self.next_char = None\r\n        self.chars = None\r\n        self.maxlen = None\r\n    \r\n    def clean_data(self, doc):\r\n        """\r\n        Accepts a single text document and performs several regex substitutions in order to clean the document. \r\n\r\n        Note\r\n        ----\r\n        Don\'t forget about online regex editors such as this one -  https://regex101.com/\r\n\r\n        Parameters\r\n        ----------\r\n        doc: string or object \r\n\r\n        Returns\r\n        -------\r\n        doc: string or object\r\n        """\r\n\r\n        # order of oper

In [94]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# a custom data prep class that we'll be using 
from data_cleaning_toolkit_class import data_cleaning_toolkit

### Use request to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) in order to learn how to download the Shakespeare Sonnets from the Gutenberg website. 

**Protip:** Do not over think it.

In [95]:
# download all of Shakespears Sonnets from the Project Gutenberg website

import requests

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use request and the url to download all of the sonnets - save the result to `r`

r = requests.get(url_shakespeare_sonnets)

In [96]:
# move the downloaded text out of the request object - save the result to `raw_text_data`
# hint: take at look at the attributes of `r`

raw_text_data = r.text

Q: what's the difference between r.text and r.content? r.content is bytes
---



In [97]:
# check the data type of `raw_text_data`
type(raw_text_data)

str

In [98]:
test = r.content
type(test)

bytes

### Data Cleaning

In [99]:
# as usual, we are tasked with cleaning up messy data
# Question: Do you see any characters that we could use to split up the text?
raw_text_data[:3000]

"\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare's Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty's rose might never die,\r\n  But as the riper

In [100]:
# split the text into lines and save the result to `split_data`

split_data = raw_text_data.split('\r\n')
split_data #type: list

["\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare",
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 "Title: Shakespeare's Sonnets",
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: April 7, 2014 [EBook #1041]',
 'Release Date: September, 1997',
 'Last Updated: March 10, 2010',
 '',
 'Language: English',
 '',
 '',
 "*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***",
 '',
 '',
 '',
 '',
 'Produced by Joseph S. Miller and Embry-Riddle Aeronautical',
 'University Library. HTML version by Al Haines.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '',
 '  I',
 '',
 '  From fairest creatures we desire increase,',
 "  That thereby beaut

In [101]:
# we need to drop all the boiler plate text (i.e. titles and descriptions) as well as white spaces
# so that we are left with only the sonnets themselves 
split_data[:20] 

["\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare",
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 "Title: Shakespeare's Sonnets",
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: April 7, 2014 [EBook #1041]',
 'Release Date: September, 1997',
 'Last Updated: March 10, 2010',
 '',
 'Language: English',
 '',
 '',
 "*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***"]

**Use list index slicing in order to remove the titles and descriptions so we are only left with the sonnets.**


In [102]:
# sonnets exists between these indicies 
# titles and descriptions exist outside of these indicies

# use index slicing to isolate the sonnet lines - save the result to `sonnets`

#Delete the first 43 lines of descriptions
del split_data[:43]


In [103]:
#Delete the last non-sonnet description

del split_data[-369:]

In [104]:
#Save the sonnet text to 'sonnets'

sonnets = split_data
sonnets

['  I',
 '',
 '  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '',
 '  II',
 '',
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '  Then being asked, where all thy beauty lies,',
 '  Where all the t

In [105]:
# notice how all non-sonnet lines have far less characters than the actual sonnet lines?
# well, let's use that observation to filter out all the non-sonnet lines
sonnets[200:240]

['  Since sweets and beauties do themselves forsake',
 '  And die as fast as they see others grow;',
 "    And nothing 'gainst Time's scythe can make defence",
 '    Save breed, to brave him when he takes thee hence.',
 '',
 '  XIII',
 '',
 '  O! that you were your self; but, love you are',
 '  No longer yours, than you your self here live:',
 '  Against this coming end you should prepare,',
 '  And your sweet semblance to some other give:',
 '  So should that beauty which you hold in lease',
 '  Find no determination; then you were',
 "  Yourself again, after yourself's decease,",
 '  When your sweet issue your sweet form should bear.',
 '  Who lets so fair a house fall to decay,',
 '  Which husbandry in honour might uphold,',
 "  Against the stormy gusts of winter's day",
 "  And barren rage of death's eternal cold?",
 '    O! none but unthrifts. Dear my love, you know,',
 '    You had a father: let your son say so.',
 '',
 '  XIV',
 '',
 '  Not from the stars do I my judgement pluck

In [106]:
# any string with less than n_chars characters will be filtered out - save results to `filtered_sonnets`

filtered_sonnets = list(filter(lambda x: len(x) > 9, sonnets))
filtered_sonnets[-100:]

['    For I have sworn thee fair, and thought thee bright,',
 '    Who art as black as hell, as dark as night.',
 '  O me! what eyes hath Love put in my head,',
 '  Which have no correspondence with true sight;',
 '  Or, if they have, where is my judgment fled,',
 '  That censures falsely what they see aright?',
 '  If that be fair whereon my false eyes dote,',
 '  What means the world to say it is not so?',
 '  If it be not, then love doth well denote',
 "  Love's eye is not so true as all men's: no,",
 "  How can it? O! how can Love's eye be true,",
 '  That is so vexed with watching and with tears?',
 '  No marvel then, though I mistake my view;',
 '  The sun itself sees not, till heaven clears.',
 "    O cunning Love! with tears thou keep'st me blind,",
 '    Lest eyes well-seeing thy foul faults should find.',
 '  Canst thou, O cruel! say I love thee not,',
 '  When I against myself with thee partake?',
 '  Do I not think on thee, when I forgot',
 '  Am of my self, all tyrant, for

In [107]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text

# import re

# def clean(text):
#     filtered = re.sub('[^a-zA-Z 0-9]', '', text)
#     filtered = filtered.lower()
#     return filtered

# filtered_sonnets = filtered_sonnets.apply(clean)
# filtered_sonnets

for i in range(len(filtered_sonnets)):
    filtered_sonnets[i] = filtered_sonnets[i].lower()
    filtered_sonnets[i] = re.sub('[^a-zA-Z 0-9]', '', filtered_sonnets[i])

filtered_sonnets

['  from fairest creatures we desire increase',
 '  that thereby beautys rose might never die',
 '  but as the riper should by time decease',
 '  his tender heir might bear his memory',
 '  but thou contracted to thine own bright eyes',
 '  feedst thy lights flame with selfsubstantial fuel',
 '  making a famine where abundance lies',
 '  thy self thy foe to thy sweet self too cruel',
 '  thou that art now the worlds fresh ornament',
 '  and only herald to the gaudy spring',
 '  within thine own bud buriest thy content',
 '  and tender churl makst waste in niggarding',
 '    pity the world or else this glutton be',
 '    to eat the worlds due by the grave and thee',
 '  when forty winters shall besiege thy brow',
 '  and dig deep trenches in thy beautys field',
 '  thy youths proud livery so gazed on now',
 '  will be a tatterd weed of small worth held',
 '  then being asked where all thy beauty lies',
 '  where all the treasure of thy lusty days',
 '  to say within thine own deep sunke

### Use custom data cleaning tool 

Use one of the methods in `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [108]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

dctk = data_cleaning_toolkit()

In [109]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`

clean_sonnets = dctk.clean_data(str(sonnets))

In [110]:
# much better!
clean_sonnets

'ifrom fairest creatures we desire increasethat thereby beautys rose might never diebut as the riper should by time deceasehis tender heir might bear his memorybut thou contracted to thine own bright eyesfeedst thy lights flame with selfsubstantial fuelmaking a famine where abundance liesthy self thy foe to thy sweet self too cruelthou that art now the worlds fresh ornamentand only herald to the gaudy springwithin thine own bud buriest thy contentand tender churl makst waste in niggardingpity the world or else this glutton beto eat the worlds due by the grave and theeiiwhen forty winters shall besiege thy browand dig deep trenches in thy beautys fieldthy youths proud livery so gazed on nowwill be a tatterd weed of small worth heldthen being asked where all thy beauty lieswhere all the treasure of thy lusty daysto say within thine own deep sunken eyeswere an alleating shame and thriftless praisehow much more praise deservd thy beautys useif thou couldst answer this fair child of minesha

In [111]:
print(type(clean_sonnets))
print(len(clean_sonnets))

<class 'str'>
88741


### Use your data tool to create character sequences 

We'll need the `create_char_sequenes` method for this task. However this method requires a parameter call `maxlen` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

In order to answer that question, let's do some statistics! 

In [123]:
def calc_stats(corpus):
    """
    Calculates statisics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnets line length - save the results to `doc_lens` 

    # use numpy to calculate and return the mean, median, std, max, min of the doc lens - all in one line of code

    doc_lens = []
    for i in range(len(corpus)):
      doc_lens.append(len(corpus[i]))

    mean = np.mean(doc_lens)
    med = np.median(doc_lens)
    std = np.std(doc_lens)
    max_ = max(doc_lens)
    min_ = min(doc_lens)

    return mean, med, std, max_, min_

In [124]:
# sonnet line length statistics 
mean ,med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(1.0, 1.0, 0.0, 1, 1)

In [125]:
# sonnet line length statistics 
mean ,med, std, max_, min_ = calc_stats(filtered_sonnets)
mean, med, std, max_, min_ 

(43.133518776077885, 43.0, 4.294747068698394, 59, 10)

In [126]:
# using the results of the sonnet line length statistics, use your judgement and select a value for maxlen
# use .create_char_sequences() to create sequences

sequences = dctk.create_char_sequenes(filtered_sonnets, maxlen=47, step=10)

sequences:  9515


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. Let's call them in the cells below.

In [127]:
# number of input features for our LSTM model
dctk.n_features

27

In [130]:
# unique charactes that appear in our sonnets 
len(dctk.chars)

27

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e. **dctk.unique_chars**) and the `number of model input features` (i.e. **dctk.n_features**) the same?

**Hint:** The model that we will shortly be building here is very similar to the text generation model that we built in the guided project.

**Answer 1:**

Every character that appears in the sonnets will be input into the LSTM model, so the number of both is the same.


**Question 2:**

Take a look at the print out of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

White space is valuable because it signals the end and beginning of words. 

----

### Use our data tool to create X and Y splits

You'll need the `create_X_and_Y` method for this task. 

In [133]:
# TODO: provide a walk through of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

# Q: why did I not have to call the object sequences in the above function? It's written in the .py object, it's referencing the self.init. Not a recommended way to architect it.

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [137]:
# notice that our input matrix isn't actually a matrix - it's a rank 3 tensor
X.shape

(9515, 47, 27)

In [136]:
y.shape

(9515, 27)

In $X$.shape we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [138]:
# first index returns a single sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False]])

Notice that each sequence (i.e. $X[i]$ where $i$ is some index value) is `maxlen` long and has `dctk.n_features` number of features. Let's try to better understand this shape. 

In [139]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(47, 27)

**Each row corresponds to a character vector** and there are `maxlen` number of character vectors. 

**Each column corresponds to a unique character** and there are `dctk.n_features` number of features. 


In [142]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True])

Notice that there is a single `TRUE` value and all the rest of the values are `FALSE`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will necessarily be a single `TRUE` value and the rest will be `FALSE`. 

Let's say that `TRUE` appears in the $ith$ index; by  $ith$ index we simply mean some index in the general case. How can we find out which character that actually corresponds to?

To answer this question, we need to use the character-to-integer look up dictionaries. 

In [143]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that that character vector is endcoding for
dctk.int_char

{0: 'k',
 1: 'o',
 2: 'h',
 3: 'a',
 4: 'q',
 5: 'e',
 6: 'r',
 7: 'c',
 8: 'x',
 9: 'l',
 10: 'f',
 11: 'd',
 12: 'p',
 13: 'g',
 14: 'n',
 15: 's',
 16: 't',
 17: 'y',
 18: 'i',
 19: 'z',
 20: 'w',
 21: 'm',
 22: 'b',
 23: 'j',
 24: 'v',
 25: 'u',
 26: ' '}

In [144]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

 
 
f
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
 
 
 
t
Sequence length: 47


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape print out of `X.shape` to a fellow classmate?


**Answer 1:**

9515, 47, 27

9515 - the number of lines there are in the corpus, which are now vectors.

47 - the length of each line, how many characters there are in it.

27 - the number of unique characters in the entire corpus, each letter represents a feature/column for one hot encoding.

----


### Build a Text Generation model

Now that we have prepped our data (and understood that process) let's finally build out our character generation model, similar to what we did in the guided project.

In [145]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunce into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next 40 chars should be that follow the seed string
    for i in range(40):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [148]:
# need this for on_epoch_end()
text = " ".join(filtered_sonnets)

In [147]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load in (the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [152]:
# build text generation model layer by layer 
# fit model

model = Sequential()
model.add(LSTM(128, input_shape=(47, 27)))
model.add(Dense(27, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y,
          batch_size=32,
          epochs=50,
          callbacks=[print_callback])

Epoch 1/50
298/298 [==============================] - 3s 5ms/step - loss: 2.8582

----- Generating text after Epoch: 0
----- Generating with seed: " thee grace   and suit thy pity like in every p"
 thee grace   and suit thy pity like in every p rfldte hhetar t cetot eguarytri prty ne
Epoch 2/50
298/298 [==============================] - 1s 5ms/step - loss: 2.5935

----- Generating text after Epoch: 1
----- Generating with seed: "one which wondrous scope affords     fair kind "
one which wondrous scope affords     fair kind  an  ied mne n med m het eremxd vptou  i
Epoch 3/50
298/298 [==============================] - 1s 5ms/step - loss: 2.3351

----- Generating text after Epoch: 2
----- Generating with seed: "ain   increasing store with loss and loss with "
ain   increasing store with loss and loss with shop ind yatt gr ot the  wiarl   wit ist
Epoch 4/50
298/298 [==============================] - 1s 5ms/step - loss: 2.2234

----- Generating text after Epoch: 3
----- Generating with seed

In [153]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's play with our trained model 

Now that we have a trained model that, though far from perfect, is able to generate actual English words, we can take a look at the predictions to continue to learn more about how a text generation model works. 

We can also take this as an opportunity to unpack the `def on_epoch_end` function to better understand how it works. 

In [154]:
# this is our joinned clean sonnet data
text

'  from fairest creatures we desire increase   that thereby beautys rose might never die   but as the riper should by time decease   his tender heir might bear his memory   but thou contracted to thine own bright eyes   feedst thy lights flame with selfsubstantial fuel   making a famine where abundance lies   thy self thy foe to thy sweet self too cruel   thou that art now the worlds fresh ornament   and only herald to the gaudy spring   within thine own bud buriest thy content   and tender churl makst waste in niggarding     pity the world or else this glutton be     to eat the worlds due by the grave and thee   when forty winters shall besiege thy brow   and dig deep trenches in thy beautys field   thy youths proud livery so gazed on now   will be a tatterd weed of small worth held   then being asked where all thy beauty lies   where all the treasure of thy lusty days   to say within thine own deep sunken eyes   were an alleating shame and thriftless praise   how much more praise des

In [157]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

11291

In [158]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e. input seqeunce into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

generated

'uring time blunt thou the lions paws   and make'

In [159]:
# this block of code let's us know what the seed string is 
# i.e. the input seqeunce into the model
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: "uring time blunt thou the lions paws   and make"
uring time blunt thou the lions paws   and make

In [160]:
# use model to predict what the next 40 chars should be that follow the seed string
for i in range(40):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [161]:
# this is the seed string
generated

'uring time blunt thou the lions paws   and make'

In [162]:
# these are the 40 chars that the model thinks should come after the seed stirng
sentence

'nd makenst reewn thy blake sweet fur thy doer r'

In [163]:
# how put it all together
generated + sentence

'uring time blunt thou the lions paws   and makend makenst reewn thy blake sweet fur thy doer r'

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN